In [ ]:
%pip install nltk spacy transformers
%pip install "numpy<2.0"
%pip install flax 
%pip install --upgrade torch
%pip install tqdm
%pip install sentence_transformers
%pip install scikit-learn
%pip install gensim
%pip install rake-nltk
%pip install ace_tools
%pip install jupyterlab
%pip install jupyterlab_widgets
%pip install concurrent
%pip install matplotlib
%pip install seaborn
%pip install kneed
%pip install transformers torch sentence-transformers

# Implementation Outline
* Text Preprocessing: Use spaCy.
* Extract Relevant Keywords
* Model Setup: Use Transformers BERT.
* Keyword Matching: Match key phrases for each maturity level.
* Scoring & Mapping: Implement a scoring function to map outputs to maturity levels.
* Visualization: Generate reports using Matplotlib or Pandas.


In [ ]:
from Keyword_Filing_Analyzer import analyze_bank_filings
import pandas as pd
import os

base_dir = "/Users/owner/Desktop/MCG/GWUxMCG/NLP Analysis/data"
output_dir = "/Users/owner/Desktop/MCG/GWUxMCG/NLP Analysis/extracted"

# Run the analysis
results = analyze_bank_filings(base_dir, output_dir)

# Load the summary data
summary_df = pd.read_csv(os.path.join(output_dir, 'keyword_summary.csv'))

# Example analyses:

# 1. Most common keywords by category
category_keywords = summary_df.explode('keywords').groupby(['category', 'keywords']).size()
print("Top keywords by category:")
for category in category_keywords.index.get_level_values(0).unique():
    print(f"\n{category}:")
    print(category_keywords[category].nlargest(10))

# 2. Keyword trends over time
time_trends = summary_df.explode('keywords').groupby(['year', 'category']).size()
print("\nKeyword usage trends by year:")
print(time_trends)

# 3. Bank comparison
bank_comparison = summary_df.groupby(['bank', 'category']).agg({
    'keywords': lambda x: len(set([item for sublist in x for item in sublist]))
})
print("\nNumber of unique keywords per bank and category:")
print(bank_comparison)


In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import json
import os

def analyze_keyword_banks(output_dir: str, min_frequency: int = 3, similarity_threshold: float = 0.7):
    """Analyze and optimize keyword banks from extraction results."""
    
    # Load the keyword frequency data
    df = pd.read_csv(os.path.join(output_dir, 'keyword_frequency.csv'))
    
    print("DataFrame columns:", df.columns)
    
    # Rename columns appropriately - adjust based on actual column names
    df.columns = ['category', 'keywords', 'frequency']
    
    # Group by category
    category_groups = df.groupby('category')
    
    optimized_banks = {}
    
    for category, group in category_groups:
        # Filter by minimum frequency
        frequent_keywords = group[group['frequency'] >= min_frequency]
        
        # Sort by frequency
        sorted_keywords = frequent_keywords.sort_values('frequency', ascending=False)
        
        # Remove near-duplicates using similarity comparison
        unique_keywords = []
        for keyword in sorted_keywords['keywords']:
            is_unique = True
            for existing in unique_keywords:
                similarity = calculate_similarity(keyword, existing)
                if similarity > similarity_threshold:
                    is_unique = False
                    break
            if is_unique:
                unique_keywords.append(keyword)
        
        # Store final keyword bank
        optimized_banks[category] = {
            'keywords': unique_keywords,
            'frequencies': sorted_keywords[sorted_keywords['keywords'].isin(unique_keywords)]['frequency'].tolist()
        }
    

    with open(os.path.join(output_dir, 'optimized_keyword_banks.json'), 'w') as f:
        json.dump(optimized_banks, f, indent=2)
        

    create_keyword_report(optimized_banks, output_dir)
    
    return optimized_banks

def calculate_similarity(kw1: str, kw2: str) -> float:
    """Calculate string similarity using character-level comparison."""
    set1 = set(kw1.lower())
    set2 = set(kw2.lower())
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    return intersection / union if union > 0 else 0

def create_keyword_report(banks: dict, output_dir: str):
    """Create a detailed report of the keyword banks."""
    report = []
    
    for category, data in banks.items():
        keywords = data['keywords']
        frequencies = data['frequencies']
        
        report.append(f"\nCategory: {category}")
        report.append(f"Total Keywords: {len(keywords)}")
        report.append(f"Average Frequency: {np.mean(frequencies):.2f}")
        report.append(f"Most Common Keywords (with frequencies):")
        
        for kw, freq in zip(keywords[:10], frequencies[:10]):
            report.append(f"  - {kw}: {freq}")
            
    with open(os.path.join(output_dir, 'keyword_bank_report.txt'), 'w') as f:
        f.write('\n'.join(report))

# Usage
output_dir = "/Users/owner/Desktop/MCG/GWUxMCG/NLP Analysis/extracted"
optimized_banks = analyze_keyword_banks(output_dir)

# Print first few keywords from each category
for category, data in optimized_banks.items():
    print(f"\n{category} top keywords:")
    for kw, freq in zip(data['keywords'][:10], data['frequencies'][:10]):
        print(f"{kw}: {freq}")

DataFrame columns: Index(['category', 'keywords', '0'], dtype='object')

Analog top keywords:
risk: 2507
management: 1794
customers: 1472
performance: 1376
compliance: 1246
market risk: 986
shareholders: 984
individuals: 927
scenarios: 654
measurement: 608

Analytics top keywords:
market: 1659
credit: 1555
analysis: 1510
sources: 1138
credit risk: 1045
risk management: 917
data: 875
reports: 772
credit ratings: 647
stress: 588

Data top keywords:
services: 1533
compliance: 1195
availability: 1106
acquisition: 1034
data: 908
security: 897
retention: 404
audit committee: 329
integrity: 320
privacy: 299

Technology top keywords:
data: 908
security: 897
technology: 891
development: 881
infrastructure: 519
governance: 398
classification: 375
cyber attacks: 369
capabilities: 310
threats: 292


In [2]:
import json

# Path to the JSON file
json_file_path = "/Users/owner/Desktop/MCG/GWUxMCG/NLP Analysis/extracted/optimized_keyword_banks.json"

# Load the JSON file
try:
    with open(json_file_path, 'r') as file:
        data_json = json.load(file)
    print("JSON file loaded successfully!")
    # Print the data structure to verify
    import pprint
    pprint.pprint(data)
except FileNotFoundError:
    print(f"Error: The file at {json_file_path} was not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    
categories = {
    "Data": [
        "information management", "data governance", "data quality", "data access", "data traceability",
        "data health", "data centers", "ata services", "data integration", "data stewardship",
        "data privacy", "data security", "data compliance", "data migration", "data modeling",
        "data warehousing", "master data management", "mdm", "data cataloging", "data retention", "data monetization",
        "information management", "data health", "data governance", "data access", "data traceability",
        "insights", "data centers", "data services", "data and analytics",'access', 'acquisition', 'architecture', 'availability', 'auto-scaling', 'auto-optimization', 'audit',
        'metadata repositories', 'federated search', 'data observability', 'semantic search',
        'integrity checks', 'validation rules', 'consistency standards', 'deduplication',
        'real-time pipelines', 'etl', 'api-based integration', 'middleware',
        'anonymization', 'pseudonymization', 'gdpr compliance', 'access logs',
        'distributed systems', 'object storage', 'file systems', 'archival solutions',
        'knowledge graphs', 'dynamic cataloging', 'operational insights', 'data democratization',
        'stewardship roles', 'audit trails', 'role based controls', 'data ethics frameworks',
        'data as a service', 'subscription models', 'ip monetization', 'marketplace ecosystems'
    ],
    "Analytics": [
        "predictive analytics", "prescriptive analytics", "descriptive analytics", "diagnostic analytics", "real-time analytics",
        "data visualization", "statistical analysis", "machine learning", "artificial intelligence", "customer analytics",
        "fraud detection", "risk analytics", "operational analytics", "marketing analytics",
        "social media analytics", "sentiment analysis", "text analytics", "churn analysis", "market basket analysis",
        "customer segmentation", "lifetime value analysis", "anomaly detection", "behavioral analytics", "analytic framework",
        "roles and skills", "analytics services", "analytics processes", "analysis complexity", "analytics",
        "capital efficiency", "create recommendations", "accuracy", "operational efficiency", "different sources",
        "personalization", "personalized", "credit decision", "fraud detection", "market data", "forecasted data",
        "actual data", "reporting", "historical data", "competitor analytics", 'predictive modeling', 'decision intelligence', 'automation in analytics',
        'behavioral insights', 'real-time optimization', 'strategic forecasts',
        'reinforcement learning', 'causality analysis', 'ensemble modeling', 'geospatial analytics',
        'journey mapping', 'propensity models', 'omnichannel behavior', 'upsell strategies',
        'workflow optimization', 'process mining', 'productivity analytics', 'capacity planning',
        'channel attribution', 'ROI measurement', 'audience profiling', 'campaign simulations',
        'anomaly scoring', 'predictive triggers', 'identity matching', 'heuristic analysis',
        'credit scoring', 'hedging models', 'stress testing', 'fiscal forecasts',
        'embedding AI', 'cloud-based dashboards', 'serverless analytics', 'visualization ecosystems'
    ],
    "Technology": [
        "cloud computing", "edge computing", "cybersecurity", "blockchain", "internet of things", "IoT"
        "artificial intelligence", "machine learning", "big data", "devops", "api management",
        "software engineering", "it infrastructure", "data encryption", "identity and access management",
        "zero trust architecture", "digital transformation", "it governance", "disaster recovery", "cloud security",
        "it infrastructure", "it development", "it support and service delivery", "software procurement", "it strategy",
        "investment in iechnology", "ai", "artificial intelligence", "machine learning", "ml", "big Data",
        "data Analytics", "genai", "generative ai", "capabilities", "cloud", "cloud Services", "technology",
        "ai-powered", "rolled out", "api", "llm", "large language model", 'quantum computing', 'neuromorphic systems', 'bioinformatics platforms', 'holographic interfaces',
        'containerization', 'microservices', 'hybrid clouds', 'devsecops',
        'robotic process automation', 'intelligent automation', 'workflow bots', 'self-healing systems',
        'intrusion detection', 'threat hunting', 'biometric access', 'penetration testing',
        'transformers', 'generative adversarial networks', 'explainable AI', 'autoML',
        'serverless computing', 'cloud-native design', 'multi-cloud orchestration', 'data lakes',
        'edge-to-cloud continuum', 'digital twins', 'virtual collaboration platforms', 'no-code/low-code platforms'
    ],
    "Analog": [
        "leadership", "employee engagement", "organizational culture", "risk management", "process automation",
        "compliance management", "audit management", "stakeholder engagement", "innovation management",
        "performance management", "strategic planning", "change management", "team collaboration",
        "crisis management", "corporate governance", "customer experience", "ethics and integrity",
        "decision-making frameworks", "workforce training", "cross-functional collaboration", "leadership", "people",
        "measures", "process optimization", "process automation", "work skills", "training Programs",
        "strategic Decisions", "managing Risks", "driving Innovation", "marketing risk", "risk Management",
        "fraud Risk", "chief data", "innovation", "virtual", "e-commerce", "governance", "stewardship",
        "cybersecurity", "privacy", "compliance", "confidentiality", "data privacy", 'growth mindset', 'transformational leadership', 'servant leadership', 'emotional intelligence',
        'diversity equity inclusion', 'psychological safety', 'learning organizations', 'agile mindsets',
        'scenario analysis', 'risk-aware culture', 'resilience planning', 'business continuity',
        'co-creation', 'knowledge-sharing ecosystems', 'cross-functional pods', 'project retrospectives',
        'value stream mapping', 'lean methodologies', 'throughput analysis', 'time-in-motion studies',
        'ESG compliance', 'ethical AI', 'sustainability goals', 'stakeholder accountability',
        'upskilling', 'reskilling', 'microlearning', 'mentorship programs',
        'net promoter score', 'personalized interactions', 'journey orchestration', 'proactive service'
    ]
}

# Update the categories dictionary with unique keywords from the JSON data
for category, data in data_json.items():
    existing_keywords = set(categories[category])
    new_keywords = set(data["keywords"])
    unique_keywords = existing_keywords.union(new_keywords)
    categories[category] = sorted(unique_keywords)  # Optionally sort the keywords


JSON file loaded successfully!
{'frequencies': [908,
                 897,
                 891,
                 881,
                 519,
                 398,
                 375,
                 369,
                 310,
                 292,
                 287,
                 168,
                 125,
                 118,
                 99,
                 92,
                 76,
                 71,
                 62,
                 59,
                 38,
                 35,
                 33,
                 30,
                 29,
                 28,
                 24,
                 21,
                 21,
                 19,
                 18,
                 17,
                 17,
                 16,
                 14,
                 14,
                 14,
                 12,
                 11,
                 11,
                 11,
                 11,
                 10,
                 9,
                 8,
            

In [3]:
len(categories['Data']) + len(categories["Analytics"]) + len(categories['Technology']) + len(categories['Analog'])

769

In [4]:
categories

{'Data': ['access',
  'access logs',
  'acquired',
  'acquisition',
  'affect availability',
  'analytics',
  'anonymity',
  'anonymization',
  'api-based integration',
  'architecture',
  'archival solutions',
  'ata services',
  'audit',
  'audit committee',
  'audit function',
  'audit premiums',
  'audit trails',
  'auditors',
  'auto-optimization',
  'auto-scaling',
  'availability',
  'availability changes',
  'availability cost',
  'availability use',
  'bank audit',
  'board audit',
  'checks',
  'company acquisition',
  'competitively significant acquisition opportunities',
  'compliance',
  'confidentiality integrity',
  'consistency',
  'consistency standards',
  'continued availability',
  'data',
  'data access',
  'data analysis',
  'data analytics capabilities',
  'data and analytics',
  'data as a service',
  'data cataloging',
  'data centers',
  'data compliance',
  'data democratization',
  'data ethics frameworks',
  'data governance',
  'data health',
  'data integ

In [5]:
import SECFilingAnalyzer
analyzer = SECFilingAnalyzer.SECFilingAnalyzer(categories=categories)

base_dir = "/Users/owner/Desktop/MCG/GWUxMCG/NLP Analysis/data"
output_dir = "/Users/owner/Desktop/MCG/GWUxMCG/NLP Analysis/results_2"

# Run the analysis
analyzer.analyze_filings(base_dir, output_dir, num_processes=4)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-28 20:45:47,830 - INFO - Process 64942 - Initializing SECFilingAnalyzer with device: cpu
2024-11-28 20:45:47,831 - INFO - Process 64942 - Initializing models...
2024-11-28 20:45:47,832 - INFO - Process 64942 - Loading MPNet for semantic analysis...
2024-11-28 20:45:47,877 - INFO - Process 64942 - Use pytorch device_name: mps
2024-11-28 20:45:47,878 - INFO - Process 64942 - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-11-28 20:45:51,588 - INFO - Process 64942 - Loading FinBERT for sentiment analysis...
2024-11-28 20:45:52,234 - INFO - Process 64942 - Computing category embeddings...
Computing category embeddings: 100%|██████████| 4/4 [00:04<00:00,  1.12s/it]
2024


✓ Saved southstate_corporation_2022_results.csv (13.8 KB)
  Files processed: 10/10


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]
2024-11-28 20:51:01,866 - INFO - Process 64942 - Saved southstate_corporation_2024 results (13.8 KB)
Processing company-years:   1%|          | 2/301 [05:05<13:16:56, 159.92s/it]2024-11-28 20:51:02,033 - INFO - Process 64942 - Processing southstate_corporation - 2023



✓ Saved southstate_corporation_2024_results.csv (13.8 KB)
  Files processed: 10/10


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.73it/s]
2024-11-28 20:54:15,695 - INFO - Process 64942 - Saved southstate_corporation_2023 results (13.8 KB)
Processing company-years:   1%|          | 3/301 [08:19<14:31:10, 175.40s/it]2024-11-28 20:54:15,861 - INFO - Process 64942 - Processing southstate_corporation - 2021


...

Batches: 100%|██████████| 1/1 [00:00<00:00, 31.13it/s]
2024-11-29 04:56:49,863 - INFO - Process 64942 - Saved jpmorgan_chase_&_co._2021 results (13.7 KB)
Processing company-years: 100%|█████████▉| 300/301 [8:10:53<01:02, 62.15s/it]2024-11-29 04:56:50,018 - INFO - Process 64942 - Processing jpmorgan_chase_&_co. - 2020



✓ Saved jpmorgan_chase_&_co._2021_results.csv (13.7 KB)
  Files processed: 10/10


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]
2024-11-29 04:57:36,842 - INFO - Process 64942 - Saved jpmorgan_chase_&_co._2020 results (13.7 KB)
Processing company-years: 100%|██████████| 301/301 [8:11:40<00:00, 98.01s/it]


✓ Saved jpmorgan_chase_&_co._2020_results.csv (13.7 KB)
  Files processed: 10/10

=== Analysis Complete ===
Total processing time: 8h 11m 40s

Results saved: 279/301 company-years
Total data size: 3.5 MB


'Analysis complete'